In [6]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import snscrape.modules.twitter as sntwitter
from textblob import TextBlob

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\Cheng
[nltk_data]     An\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

# SNSCRAPE to scrape through Twitter

In [7]:

#Creating list to append tweet data
tweets_list = []
#keyword
keyword = 'bitcoin'
# No of tweets
noOfTweet = 50000


#Loop through the usernames:

#user_names = open('News_Station.txt','r')
user_names = open('Influential_People.txt','r')

for user in user_names:
    print (user)
    
# Using TwitterSearchScraper to scrape data and append tweets to list
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper("from:"+ user +" "+keyword ).get_items()):
        if i > int(noOfTweet):
            break
        
        tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.username])




ToneVays



C:\Users\CHENGA~1\AppData\Local\Temp/ipykernel_38816/2003446335.py:22: FutureWarning: username is deprecated, use user.username instead
  tweets_list.append([tweet.date, tweet.id, tweet.content, tweet.username])


officialmcafee

VitalikButerin

aantop

TimDraper

rogerkver

elonmusk

CathieDWood

jack

michael_saylor

CobraBitcoin

ErikVoorhees

Bitboy_Crypto

VinnyLingham

adam3us

gavinandresen

NickSzabo4


# Creating and cleaning the dataframe from the tweets list above 

In [8]:
# Creating a dataframe from the tweets list above 
df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])
df['Datetime'] = pd.to_datetime(df['Datetime'],unit='ms').dt.tz_convert('Asia/Singapore')
df['Datetime'] = df['Datetime'].apply(lambda a: datetime.datetime.strftime(a,"%d-%m-%Y %H:%M:%S"))
df['Datetime'] = pd.to_datetime(df['Datetime'])          
df['Tweet Id'] = ('"'+ df['Tweet Id'].astype(str) + '"')      

# Create a function to clean the tweets
def cleanTxt(text):
    text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
    text = re.sub('#', '', text) # Removing '#' hash tag
    text = re.sub('RT[\s]+', '', text) # Removing RT
    text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
    return text

df["Text"] = df["Text"].apply(cleanTxt)


# Sentiment Analysis

NLTK

In [9]:
#Sentiment Analysis
def percentage(part,whole):
    return 100 * float(part)/float(whole)

#Iterating over the tweets in the dataframe

def apply_analysis(tweet):
    return SentimentIntensityAnalyzer().polarity_scores(tweet)


df[['neg','neu','pos','compound']] = df['Text'].apply(apply_analysis).apply(pd.Series)

def sentimental_analysis(df):
    if df['neg'] > df['pos']:
        return 'Negative'
    elif df['pos'] > df['neg']:
        return 'Positive'
    elif df['pos'] == df['neg']:
        return 'Neutral'

df['Sentiment_NLTK'] = df.apply(sentimental_analysis, axis = 1)

Textblob

In [10]:
def getSubjectivity(twt):
    return TextBlob(twt).sentiment.subjectivity
def getPolarity(twt):
    return TextBlob(twt).sentiment.polarity
def getSentiment(score):
    if score<0:
        return 'Negative'
    elif score==0:
        return 'Neutral'
    else:
        return 'Positive'
    
df['Subjectivity']=df['Text'].apply(getSubjectivity)
df['Polarity']=df['Text'].apply(getPolarity)    
df['Sentiment_TB']=df['Polarity'].apply(getSentiment)

# Generating csv

In [11]:
#df.to_csv('News_Station.csv', encoding='utf-8-sig')
df.to_csv('Influential_People.csv', encoding='utf-8-sig' ,index= False)

print('Done')

Done
